In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

def load_nifti_image(file_path):
    """
    Carica un'immagine NIfTI (.nii.gz) e restituisce i dati dell'immagine come array numpy.
    """
    img = nib.load(file_path)
    img_data = img.get_fdata()
    return img_data

def get_slice_with_largest_label(mri_path, label_path):
    """
    Carica l'immagine MRI e le label, e restituisce lo slice con la label più grande.
    """
    mri_data = load_nifti_image(mri_path)
    label_data = load_nifti_image(label_path)
    # Rendi tutti i valori != 0 uguali a 1
    label_data[label_data != 0] = 1
    # Trova lo slice con la label più grande
    slice_index = np.argmax(np.sum(label_data, axis=(0, 1)))
    
    # Restituisce lo slice dell'immagine MRI corrispondente
    mri_slice = mri_data[:, :, slice_index]
    label_slice = label_data[:, :, slice_index]
    
    return mri_slice, label_slice
def visualize_slice(mri_slice, label_slice, mri_title='MRI', label_title='Label'):
    """
    Visualizza lo slice dell'immagine MRI e delle label.
    """
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(mri_slice, cmap='gray')
    ax[0].set_title(mri_title)
    ax[1].imshow(label_slice, cmap='Reds', vmin=0, vmax=1)
    ax[1].set_title(label_title)
    plt.show()
def dice_score(mask1, mask2):
    """
    Calcola il coefficiente di Dice tra due maschere binarie.
    """
    intersection = np.sum(mask1 * mask2)
    return 2. * intersection / (np.sum(mask1) + np.sum(mask2))


In [2]:
mri_folder_path = '..\\..\\Task03_Liver\\Task03_Liver\\imagesTr\\'
label_folder_path = '..\\..\\Task03_Liver\\Task03_Liver\\labelsTr\\'

In [3]:
import segmentation_functions as sf
statistics = {
            'mean': [],
            'std': [],
            'min': [],
            'max': [],
            'area': []
        }

N=-1
for i in range(0,N+1):

    mri_path = mri_folder_path+f'liver_{i}.nii.gz'
    label_path = label_folder_path + f'liver_{i}.nii.gz'
    mri_slice, label_slice = get_slice_with_largest_label(mri_path, label_path)
    # Estrai i valori di mri_slice dove label_slice è 1
    mri_values_in_label = mri_slice[label_slice == 1]

    # Calcola le statistiche
    mean_value = np.mean(mri_values_in_label)
    std_value = np.std(mri_values_in_label)
    min_value = np.min(mri_values_in_label)
    max_value = np.max(mri_values_in_label)
    area = np.sum(label_slice == 1)
    statistics['mean'].append(mean_value)
    statistics['std'].append(std_value)
    statistics['min'].append(min_value)
    statistics['max'].append(max_value)
    statistics['area'].append(area)

# Calcola le statistiche finali
final_mean = np.mean(statistics['mean'])
final_std = np.mean(statistics['std'])
final_min = np.mean(statistics['min'])
final_max = np.mean(statistics['max'])
final_area = np.mean(statistics['area'])

# Stampa le statistiche finali
print("Final Statistics:")
print(f"Mean value: {final_mean}")
print(f"Standard deviation: {final_std}")
print(f"Min value: {final_min}")
print(f"Max value: {final_max}")
print(f"Average area: {final_area}")


Final Statistics:
Mean value: 90.03965267992405
Standard deviation: 29.46923557140283
Min value: -172.1818181818182
Max value: 237.63636363636363
Average area: 32845.72727272727


In [7]:
dice_scores_0_237 = {
    'watershed': [],
    'exploration': [],
    'flood': []
}
dice_scores_172_237 = {
    'watershed': [],
    'exploration': [],
    'flood': []
}
dice_scores_50_237 = {
    'watershed': [],
    'exploration': [],
    'flood': []
}
dice_scores_50_200 = {
    'watershed': [],
    'exploration': [],
    'flood': []
}
dice_scores_90_200 = {
    'watershed': [],
    'exploration': [],
    'flood': []
}
N=100
for i in range(N+1):
    mri_path = mri_folder_path+f'liver_{i}.nii.gz'
    label_path = label_folder_path + f'liver_{i}.nii.gz'
    mri_slice, label_slice = get_slice_with_largest_label(mri_path, label_path)
    # Estrai i valori di mri_slice dove label_slice è 1
    mri_values_in_label = mri_slice[label_slice == 1]

    wshedmask = sf.watershed_segmentation(mri_slice, tolerance=(0, 237))
    exploremask = sf.exploration_segmentation(mri_slice, label_slice, tolerance=(0, 237))
    floodmask = sf.flood_segmentation(mri_slice, label_slice, tolerance=(0, 237))

    dice_scores_0_237['watershed'].append(dice_score(wshedmask, label_slice))
    dice_scores_0_237['exploration'].append(dice_score(exploremask, label_slice))
    dice_scores_0_237['flood'].append(dice_score(floodmask, label_slice))

    wshedmask_172_237 = sf.watershed_segmentation(mri_slice, tolerance=(-172, 237))
    exploremask_172_237 = sf.exploration_segmentation(mri_slice, label_slice, tolerance=(-172, 237))
    floodmask_172_237 = sf.flood_segmentation(mri_slice, label_slice, tolerance=(-172, 237))
    dice_scores_172_237['watershed'].append(dice_score(wshedmask_172_237, label_slice))
    dice_scores_172_237['exploration'].append(dice_score(exploremask_172_237, label_slice))
    dice_scores_172_237['flood'].append(dice_score(floodmask_172_237, label_slice))

    wshedmask_50_237 = sf.watershed_segmentation(mri_slice, tolerance=(50, 237))
    exploremask_50_237 = sf.exploration_segmentation(mri_slice, label_slice, tolerance=(50, 237))
    floodmask_50_237 = sf.flood_segmentation(mri_slice, label_slice, tolerance=(50, 237))
    dice_scores_50_237['watershed'].append(dice_score(wshedmask_50_237, label_slice))
    dice_scores_50_237['exploration'].append(dice_score(exploremask_50_237, label_slice))
    dice_scores_50_237['flood'].append(dice_score(floodmask_50_237, label_slice))

    wshedmask_50_200 = sf.watershed_segmentation(mri_slice, tolerance=(50, 200))
    exploremask_50_200 = sf.exploration_segmentation(mri_slice, label_slice, tolerance=(50, 200))
    floodmask_50_200 = sf.flood_segmentation(mri_slice, label_slice, tolerance=(50, 200))
    dice_scores_50_200['watershed'].append(dice_score(wshedmask_50_200, label_slice))
    dice_scores_50_200['exploration'].append(dice_score(exploremask_50_200, label_slice))
    dice_scores_50_200['flood'].append(dice_score(floodmask_50_200, label_slice))
    wshedmask_90_200 = sf.watershed_segmentation(mri_slice, tolerance=(90, 200))
    exploremask_90_200 = sf.exploration_segmentation(mri_slice, label_slice, tolerance=(90, 200))
    floodmask_90_200 = sf.flood_segmentation(mri_slice, label_slice, tolerance=(90, 200))
    dice_scores_90_200['watershed'].append(dice_score(wshedmask_90_200, label_slice))
    dice_scores_90_200['exploration'].append(dice_score(exploremask_90_200, label_slice))
    dice_scores_90_200['flood'].append(dice_score(floodmask_90_200, label_slice))


In [10]:
print(f"Average Dice score (Watershed 0-237): {np.mean(dice_scores_0_237['watershed'])}")
print(f"Average Dice score (Exploration 0-237): {np.mean(dice_scores_0_237['exploration'])}")
print(f"Average Dice score (Flood 0-237): {np.mean(dice_scores_0_237['flood'])}")

print(f"Average Dice score (Watershed 172-237): {np.mean(dice_scores_172_237['watershed'])}")
print(f"Average Dice score (Exploration 172-237): {np.mean(dice_scores_172_237['exploration'])}")
print(f"Average Dice score (Flood 172-237): {np.mean(dice_scores_172_237['flood'])}")

print(f"Average Dice score (Watershed 50-237): {np.mean(dice_scores_50_237['watershed'])}")
print(f"Average Dice score (Exploration 50-237): {np.mean(dice_scores_50_237['exploration'])}")
print(f"Average Dice score (Flood 50-237): {np.mean(dice_scores_50_237['flood'])}")

print(f"Average Dice score (Watershed 50-200): {np.mean(dice_scores_50_200['watershed'])}")
print(f"Average Dice score (Exploration 50-200): {np.mean(dice_scores_50_200['exploration'])}")
print(f"Average Dice score (Flood 50-200): {np.mean(dice_scores_50_200['flood'])}")
print(f"Average Dice score (Watershed 90-200): {np.mean(dice_scores_90_200['watershed'])}")
print(f"Average Dice score (Exploration 90-200): {np.mean(dice_scores_90_200['exploration'])}")
print(f"Average Dice score (Flood 90-200): {np.mean(dice_scores_90_200['flood'])}")

Average Dice score (Watershed 0-237): 0.8981859051545207
Average Dice score (Exploration 0-237): 0.7449026380372065
Average Dice score (Flood 0-237): 0.8438898546796536
Average Dice score (Watershed 172-237): 0.5943178414830478
Average Dice score (Exploration 172-237): 0.5096865950674814
Average Dice score (Flood 172-237): 0.8453701821879166
Average Dice score (Watershed 50-237): 0.8457079410809787
Average Dice score (Exploration 50-237): 0.836756269277809
Average Dice score (Flood 50-237): 0.8503642660567768
Average Dice score (Watershed 50-200): 0.8664784897664655
Average Dice score (Exploration 50-200): 0.8376084855306434
Average Dice score (Flood 50-200): 0.8150959377218547
Average Dice score (Watershed 90-200): 0.5114601821865389
Average Dice score (Exploration 90-200): 0.6896569375397827
Average Dice score (Flood 90-200): 0.5602109455857386


In [4]:
dice_scores = {
    'watershed': [],
    'exploration': [],
    'flood': []
}


N=130
for i in range(101,N+1):
    mri_path = mri_folder_path+f'liver_{i}.nii.gz'
    label_path = label_folder_path + f'liver_{i}.nii.gz'
    mri_slice, label_slice = get_slice_with_largest_label(mri_path, label_path)
    # Estrai i valori di mri_slice dove label_slice è 1
    mri_values_in_label = mri_slice[label_slice == 1]

    wshedmask = sf.watershed_segmentation(mri_slice, tolerance=(50, 200))
    exploremask = sf.exploration_segmentation(mri_slice, label_slice, tolerance=(50, 200))
    floodmask = sf.flood_segmentation(mri_slice, label_slice, tolerance=(50, 200))

    dice_scores['watershed'].append(dice_score(wshedmask, label_slice))
    dice_scores['exploration'].append(dice_score(exploremask, label_slice))
    dice_scores['flood'].append(dice_score(floodmask, label_slice))

print(f"Average Dice score (Watershed): {np.mean(dice_scores['watershed'])}")
print(f"Average Dice score (Exploration): {np.mean(dice_scores['exploration'])}")
print(f"Average Dice score (Flood): {np.mean(dice_scores['flood'])}")


Average Dice score (Watershed): 0.8843652162055342
Average Dice score (Exploration): 0.8677622663760307
Average Dice score (Flood): 0.8871306349916128
